In [1]:
import socket
import time
import numpy as np
import pandas as pd

In [3]:
UPD_IP = "169.254.152.245"
UDP_PORT = 3333
print("UDP target IP:", UPD_IP)
print("UDP target port:", UDP_PORT)

UDP target IP: 169.254.152.245
UDP target port: 3333


In [4]:
# Read the chord progression data from the csv file
df = pd.read_csv("/Users/sofiavallejo/Desktop/Multiverse/chord-progressions.csv")

# Print the first 5 rows of the data
print(df.head())

  1st chord 2nd chord 3rd chord 4th chord
0         C         G        Am         F
1        Dm         G         C       NaN
2         F         C         G       NaN
3        Em        Am        Dm         G
4        Am         F         C         G


In [24]:
# Access the 1st chord column
first_chord = df['1st chord']

# Access the 2nd chord column
second_chord = df['2nd chord']

# Access the 3rd chord column
third_chord = df['3rd chord']

# Access the 4th chord column
fourth_chord = df['4th chord']

# Create a list of unique chords
chords = list(set(df['1st chord'].append(df['2nd chord']).append(df['3rd chord']).append(df['4th chord'])))

# Create a dictionary to map the chord names to integers
chord_to_int = {chord: i for i, chord in enumerate(chords)}

# Create a transition matrix
num_chords = len(chords)
transition_matrix = np.zeros((num_chords, num_chords))

# Iterate over the dataframe to populate the transition matrix
for i, row in df.iterrows():
    transition_matrix[chord_to_int[row['1st chord']], chord_to_int[row['2nd chord']]] += 1
    transition_matrix[chord_to_int[row['2nd chord']], chord_to_int[row['3rd chord']]] += 1
    transition_matrix[chord_to_int[row['3rd chord']], chord_to_int[row['4th chord']]] += 1

# Normalize the transition matrix so that it becomes a probability distribution
transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)

# Start with a random chord
current_chord = np.random.choice(num_chords)

# Create a list to store the generated chord progression
progression = [current_chord]

# Generate the next chord using the transition matrix
for i in range(100):
    current_chord = np.random.choice(num_chords, p=transition_matrix[current_chord])
    progression.append(current_chord)

# Map the generated integers back to chord names
progression = [chords[chord] for chord in progression]

# Create a dictionary to store the transition matrix
transition_matrix = {}

# Iterate through each row in the DataFrame
for i, row in df.iterrows():
    current_chord = row['1st chord']
    next_chord = row['2nd chord']
    
    # If the current chord is not already in the transition matrix, add it
    if current_chord not in transition_matrix:
        transition_matrix[current_chord] = {}
    
    # If the next chord is not already in the transition matrix for the current chord, add it
    if next_chord not in transition_matrix[current_chord]:
        transition_matrix[current_chord][next_chord] = 1
    # Otherwise, increment the count of the next chord
    else:
        transition_matrix[current_chord][next_chord] += 1

# Normalize the transition matrix
for current_chord in transition_matrix:
    total = sum(transition_matrix[current_chord].values())
    for next_chord in transition_matrix[current_chord]:
        transition_matrix[current_chord][next_chord] /= total

# Select a random initial chord
current_chord = np.random.choice(list(transition_matrix.keys()))

# Generate the chord progression
generated_progression = [current_chord]
for i in range(12): # Generate 10 chords
    if current_chord not in transition_matrix or pd.isna(current_chord):
        break
    state_distribution = list(transition_matrix[current_chord].values())
    next_chords = list(transition_matrix[current_chord].keys())
    current_chord = np.random.choice(next_chords, p=state_distribution)
    generated_progression.append(current_chord)

# Print the generated chord progression
print(generated_progression)

['Dm', 'F', 'Em', 'Am', 'F', 'Em', 'Am', 'F', 'C', 'Am', 'F', 'Em', 'Am']


/var/folders/03/znhc1cs51ld5bymktj38_fwm0000gn/T/ipykernel_4358/2791127528.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  chords = list(set(df['1st chord'].append(df['2nd chord']).append(df['3rd chord']).append(df['4th chord'])))
/var/folders/03/znhc1cs51ld5bymktj38_fwm0000gn/T/ipykernel_4358/2791127528.py:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  chords = list(set(df['1st chord'].append(df['2nd chord']).append(df['3rd chord']).append(df['4th chord'])))


In [10]:
#UPD_IP = "169.254.152.245"
#UDP_PORT = 3331

#sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
#pause = 1
#data = generated_progression

#running = False

#while True:
#    try:
#        data, addr = sock.recvfrom(3331)
#        if data == b"1":
#            running = True
#        elif data == b"0":
#            running = False
#        if running: 
#            for d in data:
#                d_bytes = d.encode('utf-8')
#                sock.sendto(d_bytes, (UPD_IP, UDP_PORT))
#                print("sent ", d)
#                time.sleep(pause)
#    except socket.error as e:
#        print("error", e)

In [25]:
UPD_IP = "169.254.152.245"
UDP_PORT = 3333

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
pause = 1
data = generated_progression

running = False
for d in data:
    d_bytes = d.encode('utf-8')
    sock.sendto(d_bytes, (UPD_IP, UDP_PORT))
    print("sent ", d)
    time.sleep(pause)

sent  Dm
sent  F
sent  Em
sent  Am
sent  F


OSError: [Errno 65] No route to host